# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [6]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [17]:
ed = Website("https://edwarddonner.com")
print(ed.links)
Markdown(ed.get_contents())

['https://edwarddonner.com/', 'https://edwarddonner.com/connect-four/', 'https://edwarddonner.com/outsmart/', 'https://edwarddonner.com/about-me-and-about-nebula/', 'https://edwarddonner.com/posts/', 'https://edwarddonner.com/', 'https://news.ycombinator.com', 'https://nebula.io/?utm_source=ed&utm_medium=referral', 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html', 'https://patents.google.com/patent/US20210049536A1/', 'https://www.linkedin.com/in/eddonner/', 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/', 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/', 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/', 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/', 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/', 'https://edwarddonner.com/2024/12/21/llm-resources-superd

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for more!
April 21, 2025
The Complete Agentic AI Engineering Course
January 23, 2025
LLM Workshop – Hands-on with Agents – resources
December 21, 2024
Welcome, SuperDataScientists!
November 13, 2024
Mastering AI and LLM Engineering – Resources
Navigation
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Get in touch
ed [at] edwarddonner [dot] com
www.edwarddonner.com
Follow me
LinkedIn
Twitter
Facebook
Subscribe to newsletter
Type your email…
Subscribe



## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [20]:
link_system_prompt = """You are provided with a list of links found on a webpage. 
You are able to decide which of the links would be most relevant to include in a brochure about the company, 
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"""
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [22]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. 
You are able to decide which of the links would be most relevant to include in a brochure about the company, 
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [23]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [24]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01/23/ll

In [25]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [26]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nari-labs/Dia-1.6B',
 '/sand-ai/MAGI-1',
 '/microsoft/bitnet-b1.58-2B-4T',
 '/ostris/Flex.2-preview',
 '/HiDream-ai/HiDream-I1-Full',
 '/models',
 '/spaces/nari-labs/Dia-1.6B',
 '/spaces/enzostvs/deepsite',
 '/spaces/InstantX/InstantCharacter',
 '/spaces/bytedance-research/UNO-FLUX',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces',
 '/datasets/Anthropic/values-in-the-wild',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets/zwhe99/DeepMath-103K',
 '/datasets/OpenGVLab/InternVL-Data',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '

In [27]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [28]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [29]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
nari-labs/Dia-1.6B
Updated
1 day ago
•
63.3k
•
1.19k
sand-ai/MAGI-1
Updated
1 day ago
•
398
microsoft/bitnet-b1.58-2B-4T
Updated
3 days ago
•
32.1k
•
798
ostris/Flex.2-preview
Updated
1 day ago
•
3.85k
•
206
HiDream-ai/HiDream-I1-Full
Updated
5 days ago
•
31.2k
•
748
Browse 1M+ models
Spaces
Running
on
Zero
713
713
Dia 1.6B
👯
Generate realistic dialogue from a script, using Dia!
Running
5.33k
5.33k
DeepSite
🐳
Generate any application with 

In [34]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [30]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [31]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnari-labs/Dia-1.6B\nUpdated\n1 day ago\n•\n63.3k\n•\n1.19k\nsand-ai/MAGI-1\nUpdated\n1 day ago\n•\n399\nmicrosoft/bitnet-b1.58-2B-4T\nUpdated\n3 days ago\n•\n32.1k\n•\n798\nostris/Flex.2-preview\nUpdated\n1 day ago\n•\n3.85k\n•\n206\nHiDream-ai/HiDream-I1-Full\nUpdated\n5 days ago\n•\n31.2k\n•\n748\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n713\n713\nDia 1.6B\n👯\nGenerate realistic dialogu

In [35]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [36]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


```markdown
# Hugging Face Company Brochure

## The AI Community Building the Future

Welcome to **Hugging Face**, the leading platform where the machine learning community collaborates on models, datasets, and applications to propel the future of artificial intelligence. Since its inception, Hugging Face has aimed to democratize AI, making powerful tools accessible to everyone.

---

## Company Overview

- **Founded:** [Year of establishment]
- **Location:** [Location]
- **Website:** [huggingface.co](https://huggingface.co)

Hugging Face serves as a robust collaboration platform, hosting over **1 million models** and **250,000 datasets**. Our users include established giants like Google, Microsoft, and Amazon, as well as individuals passionate about AI development.

---

## What We Offer

### Models and Datasets
- **Explore AI Models:** Access over 1 million models that cater to various AI and ML tasks.
- **Datasets:** Browse through 250,000+ datasets suited for all machine learning needs.

### Spaces
- **Innovative Apps:** Discover and generate applications featuring text, image, video, audio, and even 3D content.
  
### Enterprise Solutions
With industry-leading tools and enterprise solutions, our platform empowers more than **50,000 organizations** to build AI with security and dedicated support.

1. **Compute Options:** Flexible pricing starting at **$0.60/hour for GPU**.
2. **Enterprise Features:** Robust security, access controls, priority support, and more at **$20/user/month**.

---

## Company Culture

At Hugging Face, our culture thrives on collaboration, innovation, and inclusivity. We deeply value the contributions made by our community and believe in the open-source ethos to build better, share knowledge, and learn from each other. Our vibrant community fosters a collective spirit, where every idea counts.

---

## Join Us

### Careers
Hugging Face is continuously looking for passionate individuals to join our team. If you have an interest in AI/ML and want to contribute to a mission-driven company, check out our careers page for openings across various fields.

Be part of a transformative journey in AI—join us at Hugging Face, where your skills can help us shape the future of technology.

---

For more information, visit our website at [huggingface.co](https://huggingface.co) or follow us on our social channels:
- [GitHub](#)
- [Twitter](#)
- [LinkedIn](#)
- [Discord](#)

---

Together, let's build a future powered by AI!
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [37]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [38]:
stream_brochure("Marcura", "https://marcura.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://marcura.com/about'}, {'type': 'company page', 'url': 'https://marcura.com/company/data-security'}, {'type': 'company compliance', 'url': 'https://marcura.com/company/compliance'}, {'type': 'careers page', 'url': 'https://marcura.com/contact'}, {'type': 'contact page', 'url': 'https://marcura.com/contact'}, {'type': 'book a demo page', 'url': 'https://marcura.com/book-a-demo'}]}



# Marcura Brochure

## Your Partner in Maritime Operations

Marcura provides world-class maritime solutions, ensuring smooth and efficient operations for the shipping industry. We play a crucial role in managing the complexities associated with maritime operations, enabling shipping companies to focus on their core missions. 

### Why Choose Marcura?

- **Proven Expertise:** With over 20 years of experience in the industry, we support more than **950 shipping companies** and have processed over **2.5 million port calls** annually.
- **Financial Strength:** Managing **$14 billion** in payments each year and facilitating **$7 billion** in products and services, Marcura is a trusted partner for maritime financial transactions.

## Our Services

Marcura offers a suite of industry-leading services, ensuring the highest level of support in all areas of maritime operations:

- **Port Services:**  
  Optimize your port call with discounted piloting and towage, along with expert guidance for seamless operations.

- **Port Data and Analytics:**  
  Receive accurate estimates on port costs and turnaround times, allowing for informed decision-making.

- **Compliance Services:**  
  We provide thorough screening for cargo, vessel, and counterparties to ensure full regulatory compliance.

### Cutting-Edge Solutions

Introducing **Marcura AI**, our newest chapter that employs intelligent and connected AI-driven solutions, making maritime operations even more efficient and effective.

## Our Brands

Our portfolio features several market-leading brands that enhance operational efficiency:

- **DA-Desk - Disbursement Platform:**  
  A digital solution that optimizes port call cost control and enhances transparency, ensuring smooth financial management in shipping.

- **MarTrust - Maritime Payments:**  
  Streamlines crew payroll and vendor payments while maintaining compliance with international regulations, tailored specifically for the maritime industry.

## Career Opportunities at Marcura

Join a diverse and innovative team spanning **53 countries**, committed to excellence in maritime services. At Marcura, we value teamwork, integrity, and creativity, fostering a culture that encourages professional growth and development. 

### Why Work with Us?

- **Global Reach:** Collaborate with professionals across continents.
- **Dynamic Environment:** Be part of a rapidly evolving industry with the latest technology.
- **Career Growth:** Opportunities for personal and professional advancement in a supportive atmosphere.

## Connect with Us

At Marcura, we are dedicated to ensuring that maritime professionals can work smarter. For inquiries, demos, or career opportunities, please visit our website or contact us directly.

---

**Marcura**  
Your trusted ally in maritime operations.  
Visit us at [Marcura.com](https://www.marcura.com)  



In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>